# Юнит-тестирование

Разработка хороших тестов так же важна, как и разработка хорошего кода. Лучше найти ошибки самому, чем услышать о них от пользователей!

В этой лекции мы будем работать с внешними файлами. Мы сохраним наш код в файле .py, а затем сохраним скрипт тестирования в другом файле .py. Обычно такие файлы разрабатываются в текстовом редакторе - Brackets, Atom и т.д., или внутри среды разработки IDE - Spyder или Pycharm. Но поскольку мы здесь, то давайте использовать Jupyter!

Вспомним, что с помощью небольшой магии IPython мы можем записать содержимое ячейки в файл, используя `%%writefile`.<br>
И ещё кое-что, что мы не видели раньше: мы можем выполнять команды терминала (командной строки) из ячейки jupyter, используя `!`

## Инструменты тестирования

Существует много хороших библиотек для тестирования. Большинство из них - это пакеты сторонних производителей, которые требуют установки, например:

* [pylint](https://www.pylint.org/)
* [pyflakes](https://pypi.python.org/pypi/pyflakes/)
* [pep8](https://pypi.python.org/pypi/pep8)

Это простые инструменты - они смотрят на Ваш код, и сообщают о нарушении стилистики или простых проблемах. Например, если переменная используется прежде, чем она была объявлена.

Намного лучший способ протестировать Ваш код - это самим написать тесты, которые запускают Вашу программу на тестовых данных, и сравнивают фактический результат с ожидаемым результатом.<br>В стандартной библиотеке доступны два таких инструмента:

* [unittest](https://docs.python.org/3/library/unittest.html)
* [doctest](https://docs.python.org/3/library/doctest.html)

Давайте сначала посмотрим на pylint, а затем более плотно займёмся unittest.


## `pylint`

`pylint` проверяет стиль и общую логику программы.

Прежде всего, если у Вас ещё нет `pylint` ( а у Вас он наверное есть, потому что он является частью дистрибутива Anaconda ), то Вам нужно установить `pylint`.<br>После того, как это сделано, ячейку ниже можно закомментировать, она больше не нужна.

In [ ]:
! pip install pylint

Теперь сохраним в файл очень простой скрипт:

In [1]:
%%writefile simple1.py
a = 1
b = 2
print(a)
print(B)

Overwriting simple1.py


Далее проверим его с помощью pylint

In [2]:
! pylint simple1.py

************* Module simple1
C:  4, 0: Final newline missing (missing-final-newline)
C:  1, 0: Missing module docstring (missing-docstring)
C:  1, 0: Invalid constant name "a" (invalid-name)
C:  2, 0: Invalid constant name "b" (invalid-name)
E:  4, 6: Undefined variable 'B' (undefined-variable)

---------------------------------------------------------------------

Your code has been rated at -12.50/10 (previous run: 8.33/10, -20.83)





No config file found, using default configuration


Pylint сначала перечисляет замечания по стилистике - ему хочется видеть ещё одну пустую строку в конце, определения модулей и функций должны иметь описания docstrings, и простые символы - это не рекомендуемый выбор для имен переменных.

Более важно следующее - pylint нашёл ошибку в программе - переменная используется перед её объявлением. Это следует исправить.

pylint дал нашей программе оценку минус 12.5 из 10. Попробуем улучшить!

In [3]:
%%writefile simple1.py
"""
A very simple script.
"""

def myfunc():
    """
    An extremely simple function.
    """
    first = 1
    second = 2
    print(first)
    print(second)

myfunc()

Overwriting simple1.py


In [4]:
! pylint simple1.py

************* Module simple1
C: 14, 0: Final newline missing (missing-final-newline)

---------------------------------------------------------------------

Your code has been rated at 8.33/10 (previous run: -12.50/10, +20.83)





No config file found, using default configuration


Намного лучше! Теперь оценка 8.33 из 10. К сожалению, новая строка в самом конце - это особенность того, как jupyter выполняет запись в файл, и здесь мы мало что можем сделать с этим. Тем не менее, pylint помог нам найти кое-какие проблемы в коде. Но что, если проблемы будут более сложными?

In [5]:
%%writefile simple2.py
"""
A very simple script.
"""

def myfunc():
    """
    An extremely simple function.
    """
    first = 1
    second = 2
    print(first)
    print('second')

myfunc()

Overwriting simple2.py


In [6]:
! pylint simple2.py

************* Module simple2
C: 14, 0: Final newline missing (missing-final-newline)
W: 10, 4: Unused variable 'second' (unused-variable)

------------------------------------------------------------------

Your code has been rated at 6.67/10 (previous run: 6.67/10, +0.00)





No config file found, using default configuration


pylint говорит нам о неиспользуемой переменной в строке 10, но он не знает о том, что в строке 12 мы скорее всего получим не тот результат, который ожидали! Для таких тестов нам нужны более серьёзные инструменты. Пришло время рассмотреть `unittest`.

## `unittest`
`unittest` позволяет Вам создавать свои собственные программы тестирования. Цель в том, чтобы передать Вашей программе определённый набор тестовых данных, и сравнить возвращаемый результат с ожидаемым результатом. 

Создадим простой скрипт, который делает заглавной первую букву в каждом слове строки. Назовём этот скрипт **cap.py**.

In [7]:
%%writefile cap.py
def cap_text(text):
    return text.capitalize()

Overwriting cap.py


Далее напишем тестовый скрипт. Мы можем назвать его как угодно, но **test_cap.py** кажется логичным названием.

При разработке тестовых функций лучше идти от простого к сложному, поскольку эти функции будут выполняться по порядку. Здесь мы сначала проверим простые строки из одного слова, а затем строки из нескольких слов.

In [8]:
%%writefile test_cap.py
import unittest
import cap

class TestCap(unittest.TestCase):
    
    def test_one_word(self):
        text = 'python'
        result = cap.cap_text(text)
        self.assertEqual(result, 'Python')
        
    def test_multiple_words(self):
        text = 'monty python'
        result = cap.cap_text(text)
        self.assertEqual(result, 'Monty Python')
        
if __name__ == '__main__':
    unittest.main()

Overwriting test_cap.py


In [9]:
! python test_cap.py

F.
FAIL: test_multiple_words (__main__.TestCap)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "test_cap.py", line 14, in test_multiple_words
    self.assertEqual(result, 'Monty Python')
AssertionError: 'Monty python' != 'Monty Python'
- Monty python
?       ^
+ Monty Python
?       ^


----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (failures=1)


Что произошло? Оказалось, что метод `.capitalize()` переводит в верхний регистр первую букву только первого слова в строке. Если почитать описание различных методов, то мы обнаружим, что метод `.title()` кажется даст нам именно то, что нам нужно.

In [10]:
%%writefile cap.py
def cap_text(text):
    return text.title()  # replace .capitalize() with .title()

Overwriting cap.py


In [11]:
! python test_cap.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


Да, теперь тесты выполнены! Но все ли случаи мы проверили? Добавим ещё один тест в **test_cap.py** чтобы проверить, как наша программа работает со словами в апострофах, такими как *don't*.

В текстовом редакторе внести любые правки просто, однако в Jupyter придется привести весь текст заново.

In [12]:
%%writefile test_cap.py
import unittest
import cap

class TestCap(unittest.TestCase):
    
    def test_one_word(self):
        text = 'python'
        result = cap.cap_text(text)
        self.assertEqual(result, 'Python')
        
    def test_multiple_words(self):
        text = 'monty python'
        result = cap.cap_text(text)
        self.assertEqual(result, 'Monty Python')
        
    def test_with_apostrophes(self):
        text = "monty python's flying circus"
        result = cap.cap_text(text)
        self.assertEqual(result, "Monty Python's Flying Circus")
        
if __name__ == '__main__':
    unittest.main()

Overwriting test_cap.py


In [13]:
! python test_cap.py

..F
FAIL: test_with_apostrophes (__main__.TestCap)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "test_cap.py", line 19, in test_with_apostrophes
    self.assertEqual(result, "Monty Python's Flying Circus")
AssertionError: "Monty Python'S Flying Circus" != "Monty Python's Flying Circus"
- Monty Python'S Flying Circus
?              ^
+ Monty Python's Flying Circus
?              ^


----------------------------------------------------------------------
Ran 3 tests in 0.000s

FAILED (failures=1)


Вот так - теперь мы должны найти решение, которое бы учитывало апострофы! Такое решение есть - посмотрите метод `capwords` из модуля `string`. Но это упражнение мы уже оставим для Вас.

Отлично! Теперь у Вас есть базовое понимание юнит-тестирования!